In [665]:
import pandas as pd
import numpy as np
import os
import pickle

In [202]:
import numpy
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [542]:
print(os.path)

<module 'ntpath' from 'D:\\Anaconda\\lib\\ntpath.py'>


In [543]:
df = pd.read_csv('C:/Users/Alexander/Desktop/Проект/CoreClassification/Data/tablesBalanced.csv', sep=';', encoding='windows-1251', header=None)

In [544]:
df[0] = df[0].str.lower()

In [545]:
print(df[0].value_counts())

дистальный прирусловой вал                      55
проксимальный прирусловой вал                   37
отложения межрусловых площадей                  33
питающий канал                                  29
фронтальная зона проксимальной части лопасти    18
Name: 0, dtype: int64


In [546]:
X = df[1]
Y = df[0]

In [547]:
cv = CountVectorizer(lowercase=False)
cv.fit(X) # find all the unique words from the training set
X = cv.transform(X)

le = LabelEncoder()
Y_encoded = le.fit_transform(Y)
le_name_mapping = dict(zip(Y_encoded, Y))

In [548]:
print(le_name_mapping)

{0: 'дистальный прирусловой вал', 1: 'отложения межрусловых площадей', 2: 'питающий канал', 3: 'проксимальный прирусловой вал', 4: 'фронтальная зона проксимальной части лопасти'}


In [549]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_encoded, test_size=0.20, random_state=7)

In [550]:
model = xgboost.XGBClassifier(n_estimators = 128, learning_rate = 0.01, max_depth = 4)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=4, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=128, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto', ...)

In [551]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 51.43%


In [660]:
testData = ["Песчаник мелко-тонкозернистый до тонкозернистого с карбонатным и глинистокарбонатным цементом. Текстуры: массивная, горизонтально-слоистая, рябь течений, единичные прослои с деформациями, фито-детритовые импульсы."]

In [661]:
testData = cv.transform(testData)

In [662]:
probabilities = model.predict_proba(testData)

In [663]:
arr = np.empty((0, 2))
for n in range(len(probabilities[0])):
    arr = np.append(arr, np.array([[le_name_mapping[n], probabilities[0][n]]]), axis=0)

In [664]:
print(arr[arr[:,1].argsort()[::-1]])

[['питающий канал' '0.5253186']
 ['фронтальная зона проксимальной части лопасти' '0.16393165']
 ['отложения межрусловых площадей' '0.10797015']
 ['проксимальный прирусловой вал' '0.10665176']
 ['дистальный прирусловой вал' '0.096127786']]


In [667]:
filename = 'C:/Users/Alexander/Desktop/Проект/CoreClassification/Data/xgbModel.sav'
pickle.dump(model, open(filename, 'wb'))

In [669]:
print(arr[0][0])

дистальный прирусловой вал
